In [2]:
import pandas as pd
import numpy as np
import json
import os
import sys

sys.path.append("..")
from common.constants import DATAFOLDER
from preprocess.src.agg_feature import agg_feature

meta_file_path = "../data/preprocess/meta_train_test_valid.json"

f = open(meta_file_path, "r")
meta_train_test_info = json.load(f)
meta_train_test_info.keys()

dict_keys(['Bitcoin Cash', 'Binance Coin', 'Bitcoin', 'EOS.IO', 'Ethereum Classic', 'Ethereum', 'Litecoin'])

In [4]:
bitcoin_meta_info = meta_train_test_info["Bitcoin"]
bitcoin_meta_info["cpcv_folds"][0]

{'0': {'train': {'3': {'start': 1573473660, 'end': 1593043200},
   '4': {'start': 1593043260, 'end': 1612612800},
   '5': {'start': 1612612860, 'end': 1632182400}},
  'valid': {'0': {'start': 1514764860, 'end': 1534334400},
   '1': {'start': 1534334460, 'end': 1553904000}},
  'test': {'2': {'start': 1553904060, 'end': 1573473600}}},
 '1': {'train': {'1': {'start': 1534334460, 'end': 1553904000},
   '4': {'start': 1593043260, 'end': 1612612800},
   '5': {'start': 1612612860, 'end': 1632182400}},
  'valid': {'2': {'start': 1553904060, 'end': 1573473600}},
  'test': {'3': {'start': 1573473660, 'end': 1593043200}}},
 '2': {'train': {'1': {'start': 1534334460, 'end': 1553904000},
   '2': {'start': 1553904060, 'end': 1573473600},
   '5': {'start': 1612612860, 'end': 1632182400}},
  'valid': {'3': {'start': 1573473660, 'end': 1593043200}},
  'test': {'4': {'start': 1593043260, 'end': 1612612800}}},
 '3': {'train': {'1': {'start': 1534334460, 'end': 1553904000},
   '2': {'start': 1553904060, '

In [5]:
print(agg_feature(bitcoin_meta_info))

TypeError: agg_feature() missing 1 required positional argument: 'data_timestamps'

In [23]:
def sma10(df):
    hilo = (df["High"]+df["Low"]) / 2
    return df["Close"].rolling(10).mean() / hilo


asset_name = "Bitcoin"
data_file_path = os.path.join(
    DATAFOLDER.cleaned_data_root_path,
    f"{asset_name}.parquet.gzip",
)
df = pd.read_parquet(data_file_path, engine="pyarrow")
target_columns = ["Open", "High", "Low", "Close", "Volume"]

save_dir = "./_cpcv_dataset"
os.makedirs(save_dir, exist_ok=True)
save_dir = os.path.join(save_dir, asset_name)
os.makedirs(save_dir, exist_ok=True)

for cpcv_fold_idx, cpcv_fold in enumerate(bitcoin_meta_info["cpcv_folds"]):
    # _save_dir = os.path.join(save_dir, f"fold_{cpcv_fold_idx}")
    # os.makedirs(_save_dir, exist_ok=True)

    for senario_idx in cpcv_fold.keys():
        # os.makedirs(os.path.join(_save_dir, f"senario_{senario_idx}"), exist_ok=True)

        for _type in ["train", "test", "valid"]:
            # _save_dir = os.path.join(_save_dir, _type)
            _save_dir = os.path.join(save_dir, f"fold_{cpcv_fold_idx}", f"senario_{senario_idx}", _type)
            os.makedirs(_save_dir, exist_ok=True)

            for _fold_idx in cpcv_fold[senario_idx][_type]:
                start_idx, end_idx = cpcv_fold[senario_idx][_type][_fold_idx]["start"], cpcv_fold[senario_idx][_type][_fold_idx]["end"]
                _df = df.loc[start_idx:end_idx+1, target_columns]
                _df["sma20"] = sma10(df)
                save_path = os.path.join(_save_dir, f"{_fold_idx}.parquet.gzip")
                _df.to_parquet(save_path, engine="pyarrow", compression="gzip")
                
                

In [24]:
_df.head()

,Open,High,Low,Close,Volume,sma20
timestamp,,,,,,
1593043260,9289.532857,9302.4000,9271.50,9280.641429,80.786189,1.000071
1593043320,9280.170843,9294.1159,9268.16,9279.585714,122.248807,1.000685
1593043380,9279.579768,9292.2000,9268.50,9279.128571,61.963587,1.000740
1593043440,9279.057143,9292.2000,9269.60,9278.385714,62.305825,1.000648
1593043500,9278.748571,9295.4000,9270.00,9282.634048,52.254924,1.000305
